# Chatbots integration

You can find in this section some setup instructions, how to start with your first chatbot and integrate it with Facebook/Messenger or Slack.


## Facebook/Messenger setup

To configure your Messenger chatbot you need to follow the steps below.
1. Go to [Facebook for developers page](https://developers.facebook.com/).
2. Click on **My Apps** and next on **Add a New App**.
3. Go to [Facebook](https://facebook.com/) and **Create a new page**. You need to have a Facebook account. You can also do it from [Facebook for developers page](https://developers.facebook.com/). You need to choose ***Settings**, next choose **Advanced** and go to **App page** section. Click **Create New Page**.
4. Create a new **Community** page and fill out the form.
5. Go back to [Facebook for developers page](https://developers.facebook.com/) and fill out **Display Name**, **Category** and **Contact Email** in **Settings->Basic**.
6. In the **Dashboard** choose **Messenger** and click **Setup**.
7. Select **Page** in Messanger setup page and copy the **Page Access Token** below under **ACCESS TOKEN**.
8. Setup **Webhooks** and fill out the url. Use your current url if you have a public one or start the code below and use tools like [ngrok](https://ngrok.com/) to get a public url. Subscribe to **messages, message_reads**. Put a **verification token** of your choice that you need to assign to the variable **VERIFY_TOKEN** below.
9. Save Changes and you are ready to go.

In [ ]:
ACCESS_TOKEN = '' # Page Access Token
VERIFY_TOKEN = '' # Verification Token

The following code runs a Flask app that recieve two types of requests:
* GET for verification,
* POST for Facebook bot messages.

In [ ]:
from flask import Flask, request
from pymessenger.bot import Bot

app = Flask(__name__)
bot = Bot(ACCESS_TOKEN)

@app.route("/", methods=['GET', 'POST'])
def receive_message():
    if request.method == 'GET':
        token_sent = request.args.get("hub.verify_token")
        return verify_fb_token(token_sent)
    else:
        output = request.get_json()
        for event in output['entry']:            
            if 'messaging' in event:
                messaging = event['messaging']
                for message in messaging:
                    if message.get('message'):
                        recipient_id = message['sender']['id']
                        if message['message'].get('text'):
                            response_sent_text = get_message()
                            send_message(recipient_id, response_sent_text)
                return "Message Processed"

def verify_fb_token(token_sent):
    if token_sent == VERIFY_TOKEN:
        return request.args.get("hub.challenge")
    return 'Invalid verification token'

def get_message():
    response = "Hi! Here is Greg, your advisor. How can I help you?"
    return response

def send_message(recipient_id, response):
    bot.send_text_message(recipient_id, response)
    return "success"

if __name__ == "__main__":
    app.run(host="0.0.0.0",port=5000)

After the above cell is executed, you should have a message of a running flask app. Make sure you have a public url and you have provided it as a **callback url** at Facebook for developers page](https://developers.facebook.com/) **Webhook** setup page.

### Test Messenger chatbot

After you verify your app, you should get logs below the information **Running on ...** above. Now you can go to Facebook, search for your page and using the ** Send Message** blue button. This should open a chat popup and you are able to send message to your bot. Each message should be marked above as Flask logs. Please keep in mind that only you can send message as only you have the priviliges to do so. You can change it at [Facebook Developers](https://developers.facebook.com/) in the **Roles** section.

#### Messenger message types

Except of pure text messages we can also send a bit more complex messages like audio files, images or just buttons. You can find the full list of messages at [Facebook for developers page](https://developers.facebook.com/).

## Slack

Slack can be used with two API types: Real Time Messaging and Events. The first one can be used for sending messages to Slack users, groups or channels. You can also use it with other API methods that are allowed in [RTM](https://api.slack.com/rtm). Events API is used for event subscription and it means, we expect to get a message when a specific actions happens that we subscribed to.

### RTM configuration

To setup your chatbot with RTM API please follow the steps below.
1. Go to [Slack](www.slack.com) and **Register**.
2. Create a new **Workspace**.
3. Click **Browser** in the top.
4. Browser for **Bots** and choose it.
5. Click on **Add configuration** button.
6. Fill out the fields: 
    - Customize Name,
    - Full Name.
7. Copy the **API Token** and paste it into a variable SLACK_TOKEN. This variable is set below.
8. **Save Integration**.

Now, it's time to do the first test. Please keep in mind to add your bot to the channel your are sending a messsage to. The easiest way is to call the bot with @botname in the channel and send the invitation that popup.

In [ ]:
SLACK_TOKEN = ""

sc = SlackClientSlackCl (SLACK_TOKEN)

if sc.rtm_connect():
     sc.api_call("chat.postMessage", channel="#general",
                          text="Hello. I'm Anna. How can I help?", as_user=True)
else:
    print("Connection failed")

### Setup Slack for Events API

Follow the instructions to setup your Slack account for RTM API:
1. Go to your workspace and click **Build** and next **Start Building**.
2. Fill out the **App Name** and choose the Workspace. Next, click on **Create app**.
3. Go to **Event Subscriptions** and turn it on. Save changes.
4. Go to **Bot Users** and fill out the fields. You can turn the online status. Save changes.
5. Go back to **Event Subscriptions** and subscribe to Workspace Events for one of the events. It can be **reaction_added**.
6. In case of Events API we need to verify the URL that will be used for the callbacks. Use the Flask app below and  [ngrok](https://ngrok.com/). In **Request URL fill** set ngrok URL and add at the end /slack/events. Use the code below to validate the URL.
7. Go back to **Basic Information**. Fill out the App name and give an app icon (optional). Copy the **Verification Token** and copy it to the variable below. It will be used to verify our chatbot connection.
8. Go to **OAuth and Permissions** and copy the **Bot User OAuth Access Token** value. This token can be used for RTM to send messages as the same bot.
9. Go to the top and make sure you have changes on **Event Subscriptions**, **Permissions** and **Bots**.
10. Install your app.

In [ ]:
VERIFICATION_TOKEN = "" # 

In [ ]:
from flask import Flask, request
from slackeventsapi import SlackEventAdapter

app = Flask(__name__)
@app.route("/slack/events",methods=['GET','POST'])
def authorize():
  output = request.get_json()
  return output["challenge"]

if __name__ == "__main__":
  app.run(port=3000)

Now we can start a new instance of our server to handle the actions.

In [ ]:
app = Flask(__name__)

slack_events_adapter = SlackEventAdapter(VERIFICATION_TOKEN, 
                                         "/slack/events", app)

@slack_events_adapter.on("reaction_added")
def reaction_added(event):
 print("Hello")
 emoji = event.get("event").get("reaction")
 print(emoji)

if __name__ == "__main__":
  app.run(port=3000)

#### Test Slack chatbot

You can test your events or API by sending request using the [API Tester](https://api.slack.com/methods). Please keep in mind that you can combine the Events and RTM code using VERIFICATION and SLACK tokens. Changing any Events API configuration needs to be followed by an app reinstallation.